<a href="https://colab.research.google.com/github/Zarasim/building-damage-assessment/blob/main/model_training_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!git clone https://github.com/Zarasim/building-damage-assessment

Cloning into 'building-damage-assessment'...
remote: Enumerating objects: 252, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 252 (delta 59), reused 128 (delta 34), pack-reused 98 (from 1)
Receiving objects: 100% (252/252), 55.10 MiB | 39.96 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [5]:
%ls

building-damage-assessment/  drive/  sample_data/


In [6]:
%cp drive/MyDrive/Data/yolo.zip building-damage-assessment/

In [7]:
%cd building-damage-assessment

/content/building-damage-assessment


In [8]:
!pip install -r requirements.txt

Obtaining file:///content/building-damage-assessment (from -r requirements.txt (line 12))
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 646.6/646.6 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 3.9 MB/s eta 0:00

In [10]:
!nvidia-smi

Fri Sep  6 09:52:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [15]:
pip install torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 67.8 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 2.1.0
    Uninstalling triton-2.1.0:
      Successfully uninstalled triton-2.1.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.18.1
    Uninstalling nvidia-nccl-cu12-2.18.1:
      Successfully uninstalled nvidia-nccl-cu12-2.18.1
  Attempting uninstall: nvidia-cudnn-cu12
    Found existing installation: nvidia-cudnn-cu12 8.9.2.26
    Uninstalling nvidia-cudnn-cu12-8.9.2.26:
      Successfully uninstalled n

In [21]:
# check GPU is available
import torch
torch.cuda.is_available()

True

In [26]:
!unzip yolo.zip

Archive:  yolo.zip
   creating: yolo/
   creating: yolo/dataset/
   creating: yolo/dataset/images/
   creating: yolo/dataset/images/train/
  inflating: yolo/dataset/images/train/socal-fire_00000429_post_disaster.png  
  inflating: yolo/dataset/images/train/hurricane-matthew_00000089_post_disaster.png  
  inflating: yolo/dataset/images/train/socal-fire_00000253_post_disaster.png  
  inflating: yolo/dataset/images/train/guatemala-volcano_00000001_post_disaster.png  
  inflating: yolo/dataset/images/train/hurricane-florence_00000002_post_disaster.png  
  inflating: yolo/dataset/images/train/hurricane-matthew_00000155_post_disaster.png  
  inflating: yolo/dataset/images/train/santa-rosa-wildfire_00000076_post_disaster.png  
  inflating: yolo/dataset/images/train/hurricane-michael_00000258_post_disaster.png  
  inflating: yolo/dataset/images/train/hurricane-harvey_00000008_post_disaster.png  
  inflating: yolo/dataset/images/train/hurricane-florence_00000374_post_disaster.png  
  inflating:

In [33]:
import yaml
with open("yolo/dataset/data.yaml", 'r') as stream:
    try:
        config = yaml.safe_load(stream)
        config["train"] = "/content/building-damage-assessment/yolo/dataset/images/train"
        config["val"] = "/content/building-damage-assessment/yolo/dataset/images/val"
        config["test"] = "/content/building-damage-assessment/yolo/dataset/images/test"
        with open("yolo/dataset/data.yaml", 'w') as outfile:
            yaml.dump(config, outfile, default_flow_style=False)
    except yaml.YAMLError as exc:
        print(exc)

In [ ]:
!python src/yolo_damage_assessment/train_yolo.py --config config/config.yaml --output /content/drive/MyDrive/training_output

2024-09-06 10:30:33,066 - root - INFO - Starting model training
2024-09-06 10:30:33,068 - root - INFO - Loaded configuration from config/config.yaml
2024-09-06 10:30:33,068 - root - INFO - Set CUDA environment variable
2024-09-06 10:30:33,075 - root - INFO - CUDA available: True
/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py:567: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_s